## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], axis=1)

In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff_value = 500
application_types_to_replace = application_df['APPLICATION_TYPE'].value_counts()[application_df['APPLICATION_TYPE'].value_counts() < cutoff_value].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts[classification_counts > 1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff_value = 1000
classifications_to_replace = application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts() < cutoff_value].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
dummy_application_df = pd.get_dummies(application_df)
dummy_application_df.head(5)

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [10]:
# Split our preprocessed data into our features and target arrays
X = dummy_application_df.drop('IS_SUCCESSFUL', axis=1)
y = dummy_application_df['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
len(X_test_scaled)

6860

## Compile, Train and Evaluate the Model

In [15]:
# Model creation function for ease of coding later
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    activation = hp.Choice('activation', ['relu', 'tanh', 'sigmoid', 'leaky_relu', 'elu', 'selu'])

    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=120,
        step=5), activation=activation, input_dim=X_train.shape[1]))
    
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=120,
            step=5),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [13]:
# Install & import the kerastuner library
!pip install keras_tuner
import keras_tuner as kt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 6.2 MB/s eta 0:00:00


In [16]:
tuner = kt.Hyperband(
    create_model,
    objective="accuracy",
    max_epochs=100,
    hyperband_iterations=2)

In [17]:
tuner.search(X_train_scaled,y_train,epochs=10,validation_data=(X_test_scaled,y_test))

Trial 244 Complete [00h 01m 24s]
accuracy: 0.7396406531333923

Best accuracy So Far: 0.7414993047714233
Total elapsed time: 01h 31m 40s

Search: Running Trial #245

Value             |Best Value So Far |Hyperparameter
elu               |tanh              |activation
31                |56                |first_units
1                 |3                 |num_layers
66                |66                |units_0
91                |31                |units_1
6                 |26                |units_2
1                 |46                |units_3
21                |96                |units_4
91                |81                |units_5
34                |100               |tuner/epochs
0                 |34                |tuner/initial_epoch
1                 |4                 |tuner/bracket
0                 |4                 |tuner/round

Epoch 1/34
858/858 [==============================] - 3s 2ms/step - loss: 0.5803 - accuracy: 0.7150 - val_loss: 0.5739 - val_accuracy: 0.7226
Epoc

KeyboardInterrupt: ignored

In [34]:
top_hyper = tuner.get_best_hyperparameters(5)
for param in top_hyper:
    print(param.values)

{'activation': 'relu', 'first_units': 85, 'num_layers': 3, 'units_0': 70, 'units_1': 115, 'units_2': 40, 'units_3': 80, 'units_4': 85, 'tuner/epochs': 5, 'tuner/initial_epoch': 2, 'tuner/bracket': 1, 'tuner/round': 1, 'tuner/trial_id': '0009'}
{'activation': 'relu', 'first_units': 100, 'num_layers': 5, 'units_0': 70, 'units_1': 85, 'units_2': 40, 'units_3': 65, 'units_4': 80, 'tuner/epochs': 5, 'tuner/initial_epoch': 2, 'tuner/bracket': 1, 'tuner/round': 1, 'tuner/trial_id': '0012'}
{'activation': 'relu', 'first_units': 70, 'num_layers': 2, 'units_0': 35, 'units_1': 30, 'units_2': 55, 'units_3': 55, 'units_4': 35, 'tuner/epochs': 5, 'tuner/initial_epoch': 0, 'tuner/bracket': 0, 'tuner/round': 0}
{'activation': 'leaky_relu', 'first_units': 80, 'num_layers': 2, 'units_0': 40, 'units_1': 65, 'units_2': 30, 'units_3': 85, 'units_4': 115, 'tuner/epochs': 5, 'tuner/initial_epoch': 0, 'tuner/bracket': 0, 'tuner/round': 0}
{'activation': 'relu', 'first_units': 85, 'num_layers': 3, 'units_0': 7

In [16]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = X_train.shape[1]

nn = tf.keras.models.Sequential()

# Take optimized hyperparams and make a model
nn.add(tf.keras.layers.Dense(units=param.values['first_units'], activation=param.values['activation'], input_dim=X_train_scaled.shape[1]))
for i in range(param.values['num_layers']):
    nn.add(tf.keras.layers.Dense(units=param.values[f'units_{i}'], activation=param.values['activation']))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 85)                3740      
                                                                 
 dense_1 (Dense)             (None, 70)                6020      
                                                                 
 dense_2 (Dense)             (None, 115)               8165      
                                                                 
 dense_3 (Dense)             (None, 40)                4640      
                                                                 
 dense_4 (Dense)             (None, 1)                 41        
                                                                 
Total params: 22,606
Trainable params: 22,606
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [18]:
# Train the model
history = nn.fit(X_train_scaled, y_train, epochs=10, verbose=1)

Epoch 1/10
858/858 [==============================] - 7s 5ms/step - loss: 0.5663 - accuracy: 0.7228
Epoch 2/10
858/858 [==============================] - 6s 7ms/step - loss: 0.5544 - accuracy: 0.7307
Epoch 3/10
858/858 [==============================] - 6s 7ms/step - loss: 0.5517 - accuracy: 0.7316
Epoch 4/10
858/858 [==============================] - 7s 8ms/step - loss: 0.5493 - accuracy: 0.7333
Epoch 5/10
858/858 [==============================] - 5s 6ms/step - loss: 0.5483 - accuracy: 0.7336
Epoch 6/10
858/858 [==============================] - 4s 5ms/step - loss: 0.5477 - accuracy: 0.7333
Epoch 7/10
858/858 [==============================] - 3s 3ms/step - loss: 0.5466 - accuracy: 0.7346
Epoch 8/10
858/858 [==============================] - 3s 3ms/step - loss: 0.5459 - accuracy: 0.7342
Epoch 9/10
858/858 [==============================] - 2s 3ms/step - loss: 0.5453 - accuracy: 0.7354
Epoch 10/10
858/858 [==============================] - 2s 3ms/step - loss: 0.5452 - accuracy: 0.7344

In [19]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=1)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 [==============================] - 2s 7ms/step - loss: 0.5554 - accuracy: 0.7287
Loss: 0.5554087162017822, Accuracy: 0.7287172079086304


In [20]:
# Export our model to HDF5 file
#  YOUR CODE GOES HERE